In [3]:
import tensorflow as tf
import json
import numpy as np
import pandas as pd

from google.cloud import bigquery
from google.cloud import storage

from google.cloud import aiplatform

#https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom-tabular-bq-managed-dataset.ipynb

In [4]:
print(tf.__version__)

2.11.0


In [5]:
#!gsutil cp gs://smart-ml-pongthorn/New_Incident.csv data

# !gsutil -m cp -r "gs://tf1-incident-pongthorn/model" .

In [23]:
unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']
isLocalModel=True

filePath='../../data/New_Incident.csv'

modelFile_endPoint=2

localFile=True

In [24]:
# def download_table(bq_table_uri: str):
#     # Remove bq:// prefix if present
#     bqclient= bigquery.Client()
#     prefix = "bq://"
#     if bq_table_uri.startswith(prefix):
#         bq_table_uri = bq_table_uri[len(prefix) :]

#     table = bigquery.TableReference.from_string(bq_table_uri)
#     rows = bqclient.list_rows(table)
#     return rows.to_dataframe()

# dfNewData=download_table("pongthorn.SMartML.new_incident")
# dfNewData.to_csv(filePath,index=False)       
# dfNewData.info()


# Load and Map Data Test

In [6]:
if  localFile:
    dfNewData=pd.read_csv(filePath)

dfNewData=dfNewData.sample(3)
print(dfNewData.info())
# dfNewData

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 20 to 3
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3 non-null      int64  
 1   severity_id                3 non-null      int64  
 2   severity_name              3 non-null      object 
 3   sla                        3 non-null      object 
 4   product_type               3 non-null      object 
 5   brand                      3 non-null      object 
 6   service_type               3 non-null      object 
 7   incident_type              3 non-null      object 
 8   open_to_close_hour         3 non-null      float64
 9   response_to_resolved_hour  3 non-null      float64
 10  imported_at                3 non-null      object 
dtypes: float64(2), int64(2), object(7)
memory usage: 288.0+ bytes
None


In [7]:
mapping_file="incident_sevirity_to_class.json"

def download_map_severity_class_as_dict():
 
    storage_client = storage.Client()
    buckdfNewData = storage_client.bucket("smart-ml-pongthorn")
    blob = bucket.blob(mapping_file)
    blob.download_to_filename(mapping_file)
    with open(mapping_file, 'r') as file:
            return json.loads(file.read())

if  localFile==False:    
    map_sevirity_to_class= download_map_severity_class_as_dict()
else:
   with open(mapping_file, 'r') as json_file:
     map_sevirity_to_class= json.load(json_file)
                                  
                                  
print(map_sevirity_to_class)


{'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [8]:
dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)
print(dfNewData.info())
dfNewData

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 20 to 3
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3 non-null      int64  
 1   severity_id                3 non-null      int64  
 2   severity                   3 non-null      int64  
 3   severity_name              3 non-null      object 
 4   sla                        3 non-null      object 
 5   product_type               3 non-null      object 
 6   brand                      3 non-null      object 
 7   service_type               3 non-null      object 
 8   incident_type              3 non-null      object 
 9   open_to_close_hour         3 non-null      float64
 10  response_to_resolved_hour  3 non-null      float64
 11  imported_at                3 non-null      object 
dtypes: float64(2), int64(3), object(7)
memory usage: 312.0+ bytes
None


,id,severity_id,severity,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour,imported_at
20,2477,2,2,Major,24x7 4Hrs Resolution Time,Software,VMWare,Incident,Software,3.350000,3.216667,27:09.0
29,2465,3,1,Minor,24x7 4Hrs Response Time,Software,NetApp,Incident,General Incident,24.766667,24.550000,27:09.0
3,2518,2,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,7.166667,7.000000,27:09.0


In [9]:
pdPrediction=pd.DataFrame(columns=['_id','_predict_severity'])
pdPrediction

,_id,_predict_severity


# Load Model from Directory to Predict

In [ ]:
if modelFile_endPoint==1 :

    if  isLocalModel: # Window Enviroment
        PATH_FOLDER_ARTIFACTS="model"    
    else: # For running on Vertex AI Notebook t
        PATH_FOLDER_ARTIFACTS="gs://tf1-incident-pongthorn/model"

    model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
    print(f"Load from {PATH_FOLDER_ARTIFACTS}")
    # model.summary()

    
    for  row_dict in dfNewData.to_dict(orient="records"):
          incident_id=row_dict['id']
          print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
          for key_removed in unUsedColtoPredict:
           row_dict.pop(key_removed)
          # print(row_dict)  

          input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}


          predictionResult = model.predict(input_dict)
          # print(f"{predictionResult}")   
          # maxResult=np.max(arryResult)
          # maxResult=round( maxResult,4)
          # print(f"{maxResult} of {arryResult}")

          prob = tf.nn.softmax(predictionResult)
          prob_pct=(100 * prob)  
          _class = tf.argmax(predictionResult,-1).numpy()[0]
          
          pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([{'_id':incident_id, '_predict_severity':_class}])] )
          # print(input_dict)  
            
          print(f"{prob_pct} %   as {_class}")     
          print("======================================================================================")

    dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
    dfPredictData=dfPredictData.drop(columns=['_id'])
    dfPredictData=dfPredictData[['id','_predict_severity','severity','severity_name']]


# Load Model from Online EndPpint to Predict

In [12]:
# instances = []
# dfX=dfNewData.drop(columns=unUsedColtoPredict)
# for index,row in dfX.iterrows():
#     dict_item=row.to_dict()
#     instance = {}
#     for key, value in dict_item.items():
#         if key in unUsedColtoPredict:
#             continue
#         if value is None:
#             value = ""
#         instance[key] = [value]
#         # instance[key] = value
#     instances.append(instance)

# print(len(instances))
# print(instances[0])

In [14]:
PROJECT_NUMBER="pongthorn"
REGION="asia-southeast1"

In [17]:
# isEndpointAvaiable=False

# try:
#     ENDPOINT_ID="2704182877817405440"
#     endpoint = aiplatform.Endpoint(
#     endpoint_name=f"projects/{PROJECT_NUMBER}/locations/{REGION}/endpoints/{ENDPOINT_ID}")
#     print(endpoint)
#     isEndpointAvaiable=True
# except Exception as error:
#     print(str(error)
    
listRequestData=[]
if 1==1:
# if modelFile_endPoint==2 and isEndpointAvaiable:

    for data in dfNewData.to_dict(orient="records"):
        incident_id=data['id']
        severity=data['severity']
        severity_name=data['severity_name']
        #{(a if condition else b): value for key, value in dict.items()}
        request_data = {key : [value] for key,value in data.items() }
        for key_removed in unUsedColtoPredict:
           request_data.pop(key_removed)
        request_data = json.dumps(request_data)
        listRequestData.append(request_data)
        print(request_data)

#         response = endpoint.predict([request_data])
#         y_predicted = np.argmax(response.predictions, axis=1)
#         predictionResult=response[0][0]
#         print(predictionResult)
#         print(y_predicted)

#         pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([{'_id':incident_id, '_predict_severity':y_predicted[0]}])] )
#         print("======================================================================================")

#     dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
#     dfPredictData=dfPredictData.drop(columns=['_id'])
#     dfPredictData=dfPredictData[['id','_predict_severity','severity','severity_name']]


{"sla": ["24x7 4Hrs Resolution Time"], "product_type": ["Software"], "brand": ["VMWare"], "service_type": ["Incident"], "incident_type": ["Software"], "open_to_close_hour": [3.35], "response_to_resolved_hour": [3.216666667]}
{"sla": ["24x7 4Hrs Response Time"], "product_type": ["Software"], "brand": ["NetApp"], "service_type": ["Incident"], "incident_type": ["General Incident"], "open_to_close_hour": [24.76666667], "response_to_resolved_hour": [24.55]}
{"sla": ["24x7 4Hrs Resolution Time"], "product_type": ["Server"], "brand": ["HPE"], "service_type": ["Incident"], "incident_type": ["Hard Disk Drive Failure"], "open_to_close_hour": [7.166666667], "response_to_resolved_hour": [7.0]}


In [19]:
import jsonlines
with jsonlines.open('inoutput/incident_input_batch.jsonl', mode='w') as writer:
  for data in listRequestData:  
    writer.write(data)

# Load model by refering to vertext-ai registry

In [19]:
# MODEL_ID="2102389377610022912"
# MODEL_URI =  f'projects/{PROJECT_NUMBER}/locations/{REGION}/models/{MODEL_ID}'
# print(MODEL_URI)

# model = aiplatform.Model(MODEL_URI)


projects/pongthorn/locations/asia-southeast1/models/2102389377610022912


In [26]:
# !gsutil -m cp \
#   "gs://tf1-incident-pongthorn/batch_prediction/output_batch/prediction-tf1_dnn_incident_yip-2023_03_28T05_55_06_072Z/prediction.errors_stats-00000-of-00001" \
#   "gs://tf1-incident-pongthorn/batch_prediction/output_batch/prediction-tf1_dnn_incident_yip-2023_03_28T05_55_06_072Z/prediction.results-00000-of-00001" \
#   .

Copying gs://tf1-incident-pongthorn/batch_prediction/output_batch/prediction-tf1_dnn_incident_yip-2023_03_28T05_55_06_072Z/prediction.errors_stats-00000-of-00001...
Copying gs://tf1-incident-pongthorn/batch_prediction/output_batch/prediction-tf1_dnn_incident_yip-2023_03_28T05_55_06_072Z/prediction.results-00000-of-00001...
/ [0/2 files][    0.0 B/   83.0 B]   0% Done                                    
/ [0/2 files][    0.0 B/   83.0 B]   0% Done                                    
/ [1/2 files][    0.0 B/   83.0 B]   0% Done                                    
/ [2/2 files][   83.0 B/   83.0 B] 100% Done                                    

Operation completed over 2 objects/83.0 B.                                       


# Classification Rerport

In [302]:
from sklearn.metrics import confusion_matrix,classification_report


In [303]:
dfPredictData

,id,_predict_severity,severity,severity_name
0,2487,2,2,Major
1,2505,2,1,Minor
2,2490,2,2,Major
3,2517,2,2,Major
4,2516,2,1,Minor
5,2508,2,1,Minor
6,2506,2,2,Major
7,2480,2,2,Major
8,2434,2,2,Major
9,2501,2,2,Major


In [304]:
className=list(set().union(list(dfPredictData['severity'].unique()),list(dfPredictData['_predict_severity'].unique())))
actualClass=[  f'actual-{x}' for x in  className]
predictedlClass=[  f'predcited-{x}' for x in className]

y_true=list(dfPredictData['severity'])
y_pred=list(dfPredictData['_predict_severity'])
cnf_matrix = confusion_matrix(y_true,y_pred)
cnf_matrix

# #index=actual , column=prediction
cm_df = pd.DataFrame(cnf_matrix,
                     index = actualClass, 
                     columns = predictedlClass)
cm_df

,predcited-1,predcited-2
actual-1,0,3
actual-2,0,7


In [305]:

print(classification_report(y_true, y_pred, labels=className))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.70      1.00      0.82         7

    accuracy                           0.70        10
   macro avg       0.35      0.50      0.41        10
weighted avg       0.49      0.70      0.58        10



D:\ProgramData\Anaconda3\envs\dev_google\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramData\Anaconda3\envs\dev_google\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramData\Anaconda3\envs\dev_google\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
